# Find proteins within the 8p loss event (cis) that are commonly different between samples with and without the event

## Setup

In [1]:
import pandas as pd
import numpy as np
import os
import altair as alt

In [2]:
CHROMOSOME = "8"
ARM = "p"
TRANS_OR_CIS = "trans"

ttest_results_file = f"{CHROMOSOME}{ARM}_{TRANS_OR_CIS}effects_ttest.tsv"

ttest_results = pd.\
read_csv(ttest_results_file, sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

In [3]:
ttest_results

,brca_Database_ID,lscc_Database_ID,luad_Database_ID,ovarian_Database_ID,brca_pvalue,colon_pvalue,hnscc_pvalue,lscc_pvalue,luad_pvalue,ovarian_pvalue,brca_diff,colon_diff,hnscc_diff,lscc_diff,luad_diff,ovarian_diff
protein,,,,,,,,,,,,,,,,
A1BG,NP_570602.2,NP_570602.2,NP_570602.2,NP_570602,0.999623,0.935129,0.897423,0.971033,0.264654,0.839791,0.001224,-0.042624,-0.036932,0.016439,-0.457979,-0.254222
A2M,NP_000005.2,NP_000005.2|NP_001334353.1|NP_001334354.1,NP_000005.2|NP_001334353.1|NP_001334354.1|K4JD...,NP_000005,0.999039,0.989905,0.735459,0.767716,0.320054,0.866231,0.440059,0.007326,-0.119955,-0.161387,-0.546123,-0.289882
A2ML1,NP_653271.2|NP_001269353.1,NP_653271.2|NP_001269353.1,NaN,NP_653271,0.999039,NaN,0.994191,0.906043,NaN,0.936748,-0.282446,NaN,0.012262,0.233421,NaN,0.229860
AAAS,NP_056480.1|NP_001166937.1,NP_056480.1|NP_001166937.1,NP_056480.1|NP_001166937.1,NP_056480,0.999039,0.691833,0.946808,0.024379,0.467265,0.992633,0.044718,-0.084535,0.014091,0.290148,0.167781,0.017256
AACS,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_076417,0.999039,0.565289,0.946808,0.870810,0.904940,0.926544,0.177779,0.105313,0.030781,-0.095229,-0.118709,0.127899
AADAT,NP_057312.1|NP_001273611.1,NaN,NP_001273611.1|NP_001273612.1,NaN,0.999039,NaN,NaN,NaN,0.895942,NaN,-0.709031,NaN,-0.399648,NaN,-0.254802,NaN
AAED1,NP_714542.1,NP_714542.1,NP_714542.1,NaN,0.999039,NaN,0.827008,0.644550,0.704967,NaN,0.135681,NaN,0.110447,0.185691,-0.245054,NaN
AAGAB,NP_078942.3|NP_001258815.1,NP_078942.3|NP_001258814.1,NP_078942.3|NP_001258814.1,NP_078942,0.999039,0.868924,0.979517,0.619361,0.630667,0.928433,0.377825,-0.041200,-0.010700,0.134755,-0.169977,0.084824
AAK1,NP_055726.3,NP_055726.3,NaN,NP_055726,0.999039,0.932507,0.995953,0.630773,NaN,0.937577,-0.037479,0.014480,-0.001860,-0.109442,NaN,0.078751


## Reshape the input dataframe
We want to get our table to have these columns:
- cancer_type
- protein
- Database_ID
- change
- p_value

Since some cancer types have database IDs and some don't, we'll slice out and reshape the info for each cancer type individually.

In [4]:
cancer_types = sorted(ttest_results.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_results = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = ttest_results.\
    loc[:, ttest_results.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p", "change"]]
    
    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

In [5]:
long_results

,cancer_type,protein,Database_ID,adj_p,change
0,brca,A1BG,NP_570602.2,0.999623,0.001224
1,brca,A2M,NP_000005.2,0.999039,0.440059
2,brca,A2ML1,NP_653271.2|NP_001269353.1,0.999039,-0.282446
3,brca,AAAS,NP_056480.1|NP_001166937.1,0.999039,0.044718
4,brca,AACS,NP_076417.2|NP_001306769.1|NP_001306768.1,0.999039,0.177779
5,brca,AADAT,NP_057312.1|NP_001273611.1,0.999039,-0.709031
6,brca,AAED1,NP_714542.1,0.999039,0.135681
7,brca,AAGAB,NP_078942.3|NP_001258815.1,0.999039,0.377825
8,brca,AAK1,NP_055726.3,0.999039,-0.037479
9,brca,AAMDC,NP_078960.1|NP_001303886.1|NP_001303887.1,0.999039,-0.015224


## Select the proteins with a significant change, and take a detour to make some plots

In [6]:
prots = long_results[long_results["adj_p"] <= 0.05].reset_index(drop=True)
prots_cts = prots.groupby("cancer_type").count()[["protein"]]

fail_prots = long_results[long_results["adj_p"] > 0.05].reset_index(drop=True)
fail_cts = fail_prots.groupby("cancer_type").count()[["protein"]]

prots_cts.insert(0, "count_type", "Significant difference")
fail_cts.insert(0, "count_type", "No significant difference")

counts = prots_cts.append(fail_cts).sort_index().reset_index(drop=False)

alt.Chart(counts).mark_bar().encode(
    x=alt.X(
        "count_type",
        axis=alt.Axis(
            title=None,
            labels=False
        ),
        sort=["Significant difference"]
    ),
    y=alt.Y(
        "protein",
        axis=alt.Axis(
            title="Number of proteins"
        )
    ),
    color=alt.Color(
        "count_type",
        title=None,
        sort=["Significant difference"],
        scale=alt.Scale(
            domain=["Significant difference", "No significant difference"],
            range=["#2d3da4", "#d1d1d1"]
        )
    )
).facet(
    column=alt.Column(
        "cancer_type",
        title=None
    )
).properties(
    title=f"Chr {CHROMOSOME}{ARM} {TRANS_OR_CIS} effects"
).configure_title(
    anchor="middle"
)

alt.FacetChart(...)

## Find how many cancers each protein was different in

In [7]:
def make_simple_change(change_val):
    if change_val == 0:
        return 0
    if change_val > 0:
        return 1
    if change_val < 0:
        return -1

prots = prots.assign(
    simplified_change=prots["change"].apply(make_simple_change)
)

In [8]:
prots

,cancer_type,protein,Database_ID,adj_p,change,simplified_change
0,brca,CNOT8,NP_001288002.1|NP_001288003.1|NP_001288006.1|N...,0.000030,1.159894,1.0
1,brca,WRN,NP_000544.2,0.038496,-0.602335,-1.0
2,colon,AGO2,NaN,0.036205,0.176455,1.0
3,colon,ALDH1B1,NaN,0.027876,0.556227,1.0
4,colon,ARFGAP3,NaN,0.040449,-0.269368,-1.0
5,colon,ATP1B3,NaN,0.018344,0.320327,1.0
6,colon,ATP6V1A,NaN,0.013420,-0.216519,-1.0
7,colon,ATP6V1E1,NaN,0.004253,-0.256006,-1.0
8,colon,ATP6V1F,NaN,0.027876,-0.242232,-1.0
9,colon,ATP6V1G1,NaN,0.004253,-0.287800,-1.0


In [9]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist()),
    "mean_simp_change": ("simplified_change", np.mean)
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,mean_simp_change,num_cancers
protein,,,
CNOT8,"[brca, lscc, luad]",1.0,3
ATP6V1E1,"[colon, lscc, luad]",-1.0,3
ATP6V1H,"[colon, lscc, luad]",-1.0,3
WRN,"[brca, luad]",-1.0,2
AGO2,"[colon, lscc]",1.0,2
ARFGAP3,"[colon, lscc]",-1.0,2
ATP6V1G1,"[colon, lscc]",-1.0,2
CRELD2,"[colon, lscc]",-1.0,2
DDX28,"[colon, lscc]",1.0,2


In [10]:
prots_summary["num_cancers"].value_counts().sort_index(ascending=False)

3       3
2      33
1    1170
Name: num_cancers, dtype: int64

## Save results

In [11]:
prots_summary = prots_summary.assign(
    cancers=prots_summary["cancers"].apply(lambda x: "_".join(x))
)

output_file = f"pancancer_summary_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}.tsv"
prots_summary.to_csv(output_file, sep="\t")